# 0.インポート関連

In [3]:
#  000-000
##　初回　必須　　

import numpy as np
import pandas as pd
import pickle
import io
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

#jupyterlabを使う場合、この2行はいらない
from jupyterthemes import jtplot
jtplot.style(theme="monokai")


C:\Users\mk198\AppData\Local\Temp\ipykernel_5848\265227794.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
#  000-001
#　ｒａｃｅ＿ｉｎｆｏに関するコード　処理必須
ar_code_list = {\
        "10":"直",\
        "11":"左",\
        "12":"左",\
        "13":"左",\
        "14":"左",\
        "15":"左",\
        "16":"左",\
        "17":"右",\
        "18":"右",\
        "19":"右",\
        "20":"右",\
        "21":"右",\
        "22":"右",\
        "23":"左",\
        "24":"右",\
        "25":"左",\
        "26":"右",\
        "27":"左",\
        "28":"右",\
        "29":"直",\
        "51":"障",\
        "52":"障",\
        "53":"障",\
        "54":"障",\
        "55":"障",\
        "56":"障",\
        "57":"障",\
        "58":"障",\
        "59":"障"\
    } 

gr_code_list = {\
        "10":"芝",\
        "11":"芝",\
        "12":"芝",\
        "13":"芝",\
        "14":"芝",\
        "15":"芝",\
        "16":"芝",\
        "17":"芝",\
        "18":"芝",\
        "19":"芝",\
        "20":"芝",\
        "21":"芝",\
        "22":"芝",\
        "23":"ダート",\
        "24":"ダート",\
        "25":"ダート",\
        "26":"ダート",\
        "27":"サ",\
        "28":"サ",\
        "29":"ダート",\
        "51":"障害",\
        "52":"障害",\
        "53":"障害",\
        "54":"障害",\
        "55":"障害",\
        "56":"障害",\
        "57":"障害",\
        "58":"障害",\
        "59":"障害"\
    } 

class_code_list = {\
        "000":"未設定",\
        "005":"１勝クラス",\
        "010":"２勝クラス",\
        "016":"３勝クラス",\
        "701":"新馬",\
        "703":"未勝利",\
        "999":"オープン"\
    } 

grade_code_list = {\
        "A":"Ｇ１",\
        "B":"Ｇ２",\
        "C":"Ｇ３",\
        "D":"その他",\
        "E":"その他",\
        "F":"その他",\
        "G":"その他",\
        "H":"その他",\
        "L":"リステッド",\
        " ":"その他"\
    } 

juryo_shubetsu_code_list = {\
        "0":"未設定",\
        "1":"ハンデ",\
        "2":"別定",\
        "3":"馬齢",\
        "4":"定量"\
    } 

babajotai_code_list = {\
        "0":"未設定",\
        "1":"良",\
        "2":"稍重",\
        "3":"重",\
        "4":"不良"\
    } 

tenko_code_list = {\
        "0":"未設定",\
        "1":"晴",\
        "2":"曇",\
        "3":"雨",\
        "4":"小雨",\
        "5":"雪",\
        "6":"小雪"\
    } 

kyoso_shubetsu_code_list = {\
        "00":"未設定",\
        "11":"サラ２歳",\
        "12":"サラ３歳",\
        "13":"サラ３歳以上",\
        "14":"サラ４歳以上",\
        "18":"サラ障害３歳以上",\
        "19":"サラ障害４歳以上",\
        "21":"アラブ２歳",\
        "22":"アラブ３歳",\
        "23":"アラブ３歳以上",\
        "24":"アラブ４歳以上"\
    } 

keibajo_code_list = {\
        "00":"未設定",\
        "01":"札幌",\
        "02":"函館",\
        "03":"福島",\
        "04":"新潟",\
        "05":"東京",\
        "06":"中山",\
        "07":"中京",\
        "08":"京都",\
        "09":"阪神",\
        "10":"小倉"\
    } 

seibetsu_code_list = {\
        "0":"未設定",\
        "1":"牡",\
        "2":"牝",\
        "3":"セン"\
    } 


# 1.データアンロード

In [ ]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
#horse_peds = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:
# ５ｈｏｒｓｅ＿ｒｅｓｕｌｔｓデータセレクト　
horse_results = race_results[(race_results['kaisai_nen'] == '2024') & (race_results['data_kubun'] == '7')]

In [ ]:
horse_results['日付'] = horse_results['kaisai_nen'] + '/' + horse_results['kaisai_tsukihi'].str[0:2] + '/' + horse_results['kaisai_tsukihi'].str[2:4]
horse_results.replace({'keibajo_code': keibajo_code_list},inplace=True)
horse_results['kaisai_kai'] = horse_results['kaisai_kai'].astype('int')
horse_results['kaisai_kai'] = horse_results['kaisai_kai'].astype('str')
horse_results['kaisai_nichime'] = horse_results['kaisai_nichime'].astype('int')
horse_results['kaisai_nichime'] = horse_results['kaisai_nichime'].astype('str')
horse_results['開催'] = horse_results['kaisai_kai'] + horse_results['keibajo_code'] + horse_results['kaisai_nichime']
horse_results['天気'] = ''
horse_results['R'] = horse_results['race_bango'].astype('int')
horse_results['レース名'] = ''
horse_results['映像'] = '**'
horse_results['頭数'] = ''
horse_results['枠番'] = horse_results['wakuban']
horse_results['馬番'] = horse_results['umaban']
horse_results['オッズ'] = horse_results['tansho_odds'].str[0:3] + '.' + horse_results['tansho_odds'].str[3:4]
horse_results['人気'] = horse_results['tansho_ninkijun']
horse_results['着順'] = horse_results['kakutei_chakujun']
horse_results['騎手'] = horse_results['kishu_code']
horse_results['斤量'] = horse_results['futan_juryo'].str[0:2] + '.' + horse_results['futan_juryo'].str[2:3]
horse_results['距離'] = ''
horse_results['馬場'] = ''
horse_results['馬場指数'] = '**'
horse_results['タイム'] = horse_results['soha_time'].str[0:1] + ':' + horse_results['soha_time'].str[1:3] + '.' +horse_results['soha_time'].str[3:4]

horse_results['着差符号'] = horse_results['time_sa'].str[0:1]
horse_results['着差タイム'] = horse_results['time_sa'].str[1:3] + '.' + horse_results['time_sa'].str[3:4]
horse_results['着差タイム'] = horse_results['着差タイム'].astype('float')

horse_results['着差'] = horse_results.apply(\
        lambda x: x['着差タイム'] if x['着差符号'] == '+' else 0,\
                    axis=1)

horse_results['タイム指数'] = '**'
horse_results['通過'] = ''
horse_results['ペース'] = ''
horse_results['上がり'] = horse_results['kohan_3f'].str[0:2] + '.' + horse_results['kohan_3f'].str[2:3]
#horse_results['zogen_sa'] = horse_results['zogen_sa'].astype('int')
horse_results['馬体重'] = horse_results['bataiju'] + '(' + horse_results['zogen_fugo'] + horse_results['zogen_sa'] + ')'
horse_results['厩舎ｺﾒﾝﾄ'] = ''
horse_results['備考'] = ''
horse_results['勝ち馬(2着馬)'] = ''
horse_results['賞金'] = horse_results['kakutoku_honshokin']

horse_results = horse_results[['日付',\
                        '開催',\
                        '天気',\
                        'R',\
                        'レース名',\
                        '映像',\
                        '頭数',\
                        '枠番',\
                        '馬番',\
                        'オッズ',\
                        '人気',\
                        '着順',\
                        '騎手',\
                        '斤量',\
                        '距離',\
                        '馬場',\
                        '馬場指数',\
                        'タイム',\
                        '着差',\
                        'タイム指数',\
                        '通過',\
                        'ペース',\
                        '上がり',\
                        '馬体重',\
                        '厩舎ｺﾒﾝﾄ',\
                        '備考',\
                        '勝ち馬(2着馬)',\
                        '賞金']]
horse_results = horse_results.rename_axis(None)
#horse_results.head()


In [ ]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
#race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
horse_info = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:
#４
horse_info['生年月日'] = pd.to_datetime(horse_info["seinengappi"],format='%Y%m%d')
horse_info['調教師'] = horse_info['chokyoshimei_ryakusho']
horse_info['馬主'] = horse_info['banushimei']
horse_info['生産者'] = horse_info['seisanshamei']
horse_info['産地'] = horse_info['sanchimei']
horse_info['セリ取引価格'] = ''
horse_info['獲得賞金'] = horse_info['heichi_honshokin_ruikei'].astype('int')
horse_info['獲得賞金'] = horse_info['獲得賞金'] / 100
horse_info['通算成績'] = ''
horse_info['主な勝鞍'] = ''
horse_info['近親馬'] = ''
horse_info['trainer_id'] = horse_info['chokyoshi_code']
horse_info['owner_id'] = horse_info['banushi_code']
horse_info['breeder_id'] = horse_info['seisansha_code']
horse_info['募集情報'] = ''

horse_info = horse_info.set_index('ketto_toroku_bango')
horse_info = horse_info[['生年月日',\
                        '調教師',\
                        '馬主',\
                        '生産者',\
                        '産地',\
                        'セリ取引価格',\
                        '獲得賞金',\
                        '通算成績',\
                        '主な勝鞍',\
                        '近親馬',\
                        'trainer_id',\
                        'owner_id',\
                        'breeder_id',\
                        '募集情報']]
horse_info = horse_info.rename_axis(None)


In [ ]:
horse_info.head()

In [ ]:
#２
###　初回　必須　　0010-10
### アンロード
race_info = pd.read_pickle('../data/race_info.pickle')
#race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
#horse_peds = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:
race_info['race_class'].value_counts()
#pd.set_option('display.max_columns', 100)
#race_info.head()

In [ ]:
race_info['course_len'] = race_info['kyori']
race_info['around'] = race_info['track_code']
race_info.replace({'around': ar_code_list},inplace=True)
race_info.replace({'tenko_code': tenko_code_list},inplace=True)
race_info['weather'] = race_info['tenko_code']
race_info['race_type'] = race_info['track_code']
race_info.replace({'race_type': gr_code_list},inplace=True)
race_info.replace({'race_class': class_code_list},inplace=True)
race_info['ground_state'] = race_info.apply(\
        lambda x: x['babajotai_code_shiba'] if x['race_type'] == '芝' else x['babajotai_code_dirt'] ,\
                    axis=1)
race_info.replace({'ground_state': babajotai_code_list},inplace=True)
race_info['race_class'] = race_info['kyoso_joken_code']
race_info.replace({'race_class': class_code_list},inplace=True)
race_info.replace({'grade_code': grade_code_list},inplace=True)
race_info['race_class'] = race_info.apply(\
        lambda x: x['race_class'] if x['grade_code'] == 'その他' else x['grade_code'] ,\
                    axis=1)
race_info['date2'] = race_info['kaisai_nen'] + race_info['kaisai_tsukihi']
race_info['date'] = pd.to_datetime(race_info["date2"],format='%Y%m%d')
#race_info['産地'] = race_info['sanchimei']
race_info = race_info[['course_len',\
                        'around',\
                        'weather',\
                        'race_type',\
                        'ground_state',\
                        'date',\
                        'race_class']]
race_info = race_info.rename_axis(None)



In [ ]:
race_info.head()

In [ ]:
#　５続き
horse_results = pd.merge(horse_results,race_info,left_index=True,right_index=True,how='left')

horse_results['天気'] = horse_results['weather']
horse_results['距離'] = horse_results['course_len']
horse_results['馬場'] = horse_results['ground_state']

horse_results = horse_results[['日付',\
                        '開催',\
                        '天気',\
                        'R',\
                        'レース名',\
                        '映像',\
                        '頭数',\
                        '枠番',\
                        '馬番',\
                        'オッズ',\
                        '人気',\
                        '着順',\
                        '騎手',\
                        '斤量',\
                        '距離',\
                        '馬場',\
                        '馬場指数',\
                        'タイム',\
                        '着差',\
                        'タイム指数',\
                        '通過',\
                        'ペース',\
                        '上がり',\
                        '馬体重',\
                        '厩舎ｺﾒﾝﾄ',\
                        '備考',\
                        '勝ち馬(2着馬)',\
                        '賞金']]


In [ ]:
horse_results.head()

In [ ]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
#horse_peds = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:
# １ｒｅｓｕｌｔｓデータセレクト　
results = race_results[(race_results['kaisai_nen'] == '2024') & (race_results['data_kubun'] == '7')]

In [ ]:

###
results['着順'] = results['kakutei_chakujun']
results['枠番'] = results['wakuban']
results['馬番'] = results['umaban']
results['馬名'] = results['bamei']
results.replace({'seibetsu_code': seibetsu_code_list},inplace=True)
results['馬齢'] = results['barei']
results['性齢'] = results['seibetsu_code'] + results['barei']
results['斤量'] = results['futan_juryo'].str[0:2] + '.' + results['futan_juryo'].str[2:3]
results['騎手'] = results['kishu_code']
results['タイム'] = results['soha_time'].str[0:1] + ':' + results['soha_time'].str[1:3] + '.' +results['soha_time'].str[3:4]
results['着差符号'] = results['time_sa'].str[0:1]
results['着差タイム'] = results['time_sa'].str[1:3] + '.' + results['time_sa'].str[3:4]
results['着差タイム'] = results['着差タイム'].astype('float')

results['着差'] = results.apply(\
        lambda x: x['着差タイム'] if x['着差符号'] == '+' else 0,\
                    axis=1)
results['単勝'] = results['tansho_odds'].str[0:3] + '.' + results['tansho_odds'].str[3:4]
results['人気'] = results['tansho_ninkijun']
results['馬体重'] = results['bataiju'] + '(' + results['zogen_fugo'] + results['zogen_sa'] + ')'
results['調教師'] = results['chokyoshi_code']
results['horse_id'] = results['ketto_toroku_bango']
results['jockey_id'] = results['kishu_code']
results['trainer_id'] = results['chokyoshi_code']
results['owner_id'] = results['banushi_code']

###

results = results[['着順',\
                '枠番',\
                '馬番',\
                '馬名',\
                '性齢',\
                '斤量',\
                '騎手',\
                '着差',\
                '単勝',\
                '人気',\
                '馬体重',\
                '調教師',\
                'horse_id',\
                'jockey_id',\
                'trainer_id',\
                'owner_id']]
results = results.rename_axis(None)
results.head()


In [ ]:
results.head()

In [ ]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
#race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
peds = pd.read_pickle('../data/horse_peds.pickle')
#race_return = pd.read_pickle('../data/race_return.pickle')

In [ ]:

peds = peds.reset_index()
peds = peds.set_index('ketto_toroku_bango')
peds['peds_0'] = peds['ketto_joho_01a']
peds['peds_1'] = peds['ketto_joho_02a']
peds['peds_2'] = peds['ketto_joho_03a']
peds['peds_3'] = peds['ketto_joho_04a']
peds['peds_4'] = peds['ketto_joho_05a']
peds['peds_5'] = peds['ketto_joho_06a']
peds['peds_6'] = peds['ketto_joho_07a']
peds['peds_7'] = peds['ketto_joho_08a']
peds['peds_8'] = peds['ketto_joho_09a']
peds['peds_9'] = peds['ketto_joho_10a']
peds['peds_10'] = peds['ketto_joho_11a']
peds['peds_11'] = peds['ketto_joho_12a']
peds['peds_12'] = peds['ketto_joho_13a']
peds['peds_13'] = peds['ketto_joho_14a']

peds = peds[[\
                'peds_0',\
                'peds_1',\
                'peds_2',\
                'peds_3',\
                'peds_4',\
                'peds_5',\
                'peds_6',\
                'peds_7',\
                'peds_8',\
                'peds_9',\
                'peds_10',\
                'peds_11',\
                'peds_12',\
                'peds_13'\
                ]]
peds = peds.rename_axis(None)
peds.head()


In [ ]:
peds.head()

In [18]:
#000-010
###　初回　必須　　0010-10
### アンロード
#race_info = pd.read_pickle('../data/race_info.pickle')
#race_results = pd.read_pickle('../data/race_results.pickle')
#horse_info = pd.read_pickle('../data/horse_info.pickle')
#peds = pd.read_pickle('../data/horse_peds.pickle')
race_return = pd.read_pickle('../data/race_return.pickle')

In [19]:
#単勝
race_return_1 = race_return.copy()
race_return_1['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_1['0'] = '単勝'
race_return_1['1'] = race_return['haraimodoshi_tansho_1a'].astype('int')
race_return_1['2'] = race_return['haraimodoshi_tansho_1b'].astype('int')
#race_return_1['3'] = race_return['haraimodoshi_tansho_1c']
race_return_1 = race_return_1.reset_index()
race_return_1 = race_return_1.set_index('key')

race_return_1 = race_return_1[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_1 = race_return_1.rename_axis(None)



In [20]:
#複勝
race_return_2 = race_return.copy()
race_return_2['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_2['0'] = '複勝'
race_return_2['haraimodoshi_fukusho_1a'] = race_return_2['haraimodoshi_fukusho_1a'].astype('int')
race_return_2['haraimodoshi_fukusho_1a'] = race_return_2['haraimodoshi_fukusho_1a'].astype('str')
race_return_2['haraimodoshi_fukusho_2a'] = race_return_2['haraimodoshi_fukusho_2a'].astype('int')
race_return_2['haraimodoshi_fukusho_2a'] = race_return_2['haraimodoshi_fukusho_2a'].astype('str')
race_return_2['haraimodoshi_fukusho_3a'] = race_return_2['haraimodoshi_fukusho_3a'].astype('int')
race_return_2['haraimodoshi_fukusho_3a'] = race_return_2['haraimodoshi_fukusho_3a'].astype('str')
race_return_2['haraimodoshi_fukusho_1b'] = race_return_2['haraimodoshi_fukusho_1b'].astype('int')
race_return_2['haraimodoshi_fukusho_1b'] = race_return_2['haraimodoshi_fukusho_1b'].astype('str')
race_return_2['haraimodoshi_fukusho_2b'] = race_return_2['haraimodoshi_fukusho_2b'].astype('int')
race_return_2['haraimodoshi_fukusho_2b'] = race_return_2['haraimodoshi_fukusho_2b'].astype('str')
race_return_2['haraimodoshi_fukusho_3b'] = race_return_2.apply(\
        lambda x: x['haraimodoshi_fukusho_3b'] if x['haraimodoshi_fukusho_3b'] != '         ' else '0000000000',\
                    axis=1)
race_return_2['haraimodoshi_fukusho_3b'] = race_return_2['haraimodoshi_fukusho_3b'].astype('int')
race_return_2['haraimodoshi_fukusho_3b'] = race_return_2['haraimodoshi_fukusho_3b'].astype('str')


race_return_2['1'] = race_return_2['haraimodoshi_fukusho_1a'] + 'br' + race_return_2['haraimodoshi_fukusho_2a'] + 'br' + race_return_2['haraimodoshi_fukusho_3a']
race_return_2['2'] = race_return_2['haraimodoshi_fukusho_1b'] + 'br' + race_return_2['haraimodoshi_fukusho_2b'] + 'br' + race_return_2['haraimodoshi_fukusho_3b']
#race_return_2['3'] = race_return['haraimodoshi_tansho_1c']
race_return_2 = race_return_2.reset_index()
race_return_2 = race_return_2.set_index('key')

race_return_2 = race_return_2[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_2 = race_return_2.rename_axis(None)

In [ ]:

race_return_2.head()
#race_return['haraimodoshi_fukusho_3b'].value_counts()


In [21]:
#馬連
race_return_3 = race_return.copy()
race_return_3['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_3['0'] = '馬連'
race_return_3['wk_1'] = race_return_3['haraimodoshi_umaren_1a'].str[0:2]
race_return_3['wk_1'] = race_return_3['wk_1'].astype('int')
race_return_3['wk_1'] = race_return_3['wk_1'].astype('str')
race_return_3['wk_2'] = race_return_3['haraimodoshi_umaren_1a'].str[2:4]
race_return_3['wk_2'] = race_return_3['wk_2'].astype('int')
race_return_3['wk_2'] = race_return_3['wk_2'].astype('str')
race_return_3['1'] = race_return_3['wk_1'] + ' - ' + race_return_3['wk_2']
race_return_3['2'] = race_return_3['haraimodoshi_umaren_1b'].astype('int')
#race_return_1['3'] = race_return['haraimodoshi_umaren_1c']
race_return_3 = race_return_3.reset_index()
race_return_3 = race_return_3.set_index('key')

race_return_3 = race_return_3[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_3 = race_return_3.rename_axis(None)



In [ ]:
race_return_3.head()

In [22]:
#wide
race_return_4 = race_return.copy()
race_return_4['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_4['0'] = 'ワイド'
#race_return_4['haraimodoshi_wide_1a'] = race_return_4['haraimodoshi_wide_1a'].astype('int')
#race_return_4['haraimodoshi_wide_1a'] = race_return_4['haraimodoshi_wide_1a'].astype('str')
#race_return_4['haraimodoshi_wide_2a'] = race_return_4['haraimodoshi_wide_2a'].astype('int')
#race_return_4['haraimodoshi_wide_2a'] = race_return_4['haraimodoshi_wide_2a'].astype('str')
#race_return_4['haraimodoshi_wide_3a'] = race_return_4['haraimodoshi_wide_3a'].astype('int')
#race_return_4['haraimodoshi_wide_3a'] = race_return_4['haraimodoshi_wide_3a'].astype('str')
race_return_4['wk_1'] = race_return_4['haraimodoshi_wide_1a'].str[0:2]
race_return_4['wk_1'] = race_return_4['wk_1'].astype('int')
race_return_4['wk_1'] = race_return_4['wk_1'].astype('str')
race_return_4['wk_2'] = race_return_4['haraimodoshi_wide_1a'].str[2:4]
race_return_4['wk_2'] = race_return_4['wk_2'].astype('int')
race_return_4['wk_2'] = race_return_4['wk_2'].astype('str')
race_return_4['wk_11'] = race_return_4['wk_1'] + ' - ' + race_return_4['wk_2']

race_return_4['wk_1'] = race_return_4['haraimodoshi_wide_2a'].str[0:2]
race_return_4['wk_1'] = race_return_4['wk_1'].astype('int')
race_return_4['wk_1'] = race_return_4['wk_1'].astype('str')
race_return_4['wk_2'] = race_return_4['haraimodoshi_wide_2a'].str[2:4]
race_return_4['wk_2'] = race_return_4['wk_2'].astype('int')
race_return_4['wk_2'] = race_return_4['wk_2'].astype('str')
race_return_4['wk_21'] = race_return_4['wk_1'] + ' - ' + race_return_4['wk_2']

race_return_4['wk_1'] = race_return_4['haraimodoshi_wide_3a'].str[0:2]
race_return_4['wk_1'] = race_return_4['wk_1'].astype('int')
race_return_4['wk_1'] = race_return_4['wk_1'].astype('str')
race_return_4['wk_2'] = race_return_4['haraimodoshi_wide_3a'].str[2:4]
race_return_4['wk_2'] = race_return_4['wk_2'].astype('int')
race_return_4['wk_2'] = race_return_4['wk_2'].astype('str')
race_return_4['wk_31'] = race_return_4['wk_1'] + ' - ' + race_return_4['wk_2']

race_return_4['haraimodoshi_wide_1b'] = race_return_4['haraimodoshi_wide_1b'].astype('int')
race_return_4['haraimodoshi_wide_1b'] = race_return_4['haraimodoshi_wide_1b'].astype('str')
race_return_4['haraimodoshi_wide_2b'] = race_return_4['haraimodoshi_wide_2b'].astype('int')
race_return_4['haraimodoshi_wide_2b'] = race_return_4['haraimodoshi_wide_2b'].astype('str')
race_return_4['haraimodoshi_wide_3b'] = race_return_4.apply(\
        lambda x: x['haraimodoshi_wide_3b'] if x['haraimodoshi_wide_3b'] != '         ' else '0000000000',\
                    axis=1)
race_return_4['haraimodoshi_wide_3b'] = race_return_4['haraimodoshi_wide_3b'].astype('int')
race_return_4['haraimodoshi_wide_3b'] = race_return_4['haraimodoshi_wide_3b'].astype('str')

race_return_4['1'] = race_return_4['wk_11'] + 'br' + race_return_4['wk_21'] + 'br' + race_return_4['wk_31']
race_return_4['2'] = race_return_4['haraimodoshi_wide_1b'] + 'br' + race_return_4['haraimodoshi_wide_2b'] + 'br' + race_return_4['haraimodoshi_wide_3b']
#race_return_4['3'] = race_return['haraimodoshi_tansho_1c']
race_return_4 = race_return_4.reset_index()
race_return_4 = race_return_4.set_index('key')

race_return_4 = race_return_4[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_4 = race_return_4.rename_axis(None)

In [11]:
race_return_4.head()

,0,1,2
201806010101,ワイド,3 - 15br6 - 15br3 - 6,270br350br620
201806010102,ワイド,9 - 14br2 - 14br2 - 9,9960br1960br32590
201806010103,ワイド,2 - 7br7 - 11br2 - 11,2580br2770br420
201806010104,ワイド,4 - 6br4 - 10br6 - 10,580br920br590
201806010105,ワイド,3 - 9br2 - 9br2 - 3,290br980br2940


In [23]:
#馬連
race_return_5 = race_return.copy()
race_return_5['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_5['0'] = '３連複'
race_return_5['wk_1'] = race_return_5['haraimodoshi_sanrenpuku_1a'].str[0:2]
race_return_5['wk_1'] = race_return_5['wk_1'].astype('int')
race_return_5['wk_1'] = race_return_5['wk_1'].astype('str')
race_return_5['wk_2'] = race_return_5['haraimodoshi_sanrenpuku_1a'].str[2:4]
race_return_5['wk_2'] = race_return_5['wk_2'].astype('int')
race_return_5['wk_2'] = race_return_5['wk_2'].astype('str')
race_return_5['wk_3'] = race_return_5['haraimodoshi_sanrenpuku_1a'].str[4:6]
race_return_5['wk_3'] = race_return_5['wk_3'].astype('int')
race_return_5['wk_3'] = race_return_5['wk_3'].astype('str')
race_return_5['1'] = race_return_5['wk_1'] + ' - ' + race_return_5['wk_2'] + ' - ' + race_return_5['wk_3']
race_return_5['2'] = race_return_5['haraimodoshi_sanrenpuku_1b'].astype('int')
#race_return_1['3'] = race_return['haraimodoshi_sanrenpuku_1c']
race_return_5 = race_return_5.reset_index()
race_return_5 = race_return_5.set_index('key')

race_return_5 = race_return_5[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_5 = race_return_5.rename_axis(None)




In [13]:
race_return_5.head()

,0,1,2
201806010101,馬連,3 - 6 - 15,1290
201806010102,馬連,2 - 9 - 14,277720
201806010103,馬連,2 - 7 - 11,11350
201806010104,馬連,4 - 6 - 10,2540
201806010105,馬連,2 - 3 - 9,4850


In [24]:
#馬連
race_return_6 = race_return.copy()
race_return_6['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_6['0'] = '３連単'
race_return_6['wk_1'] = race_return_6['haraimodoshi_sanrentan_1a'].str[0:2]
race_return_6['wk_1'] = race_return_6['wk_1'].astype('int')
race_return_6['wk_1'] = race_return_6['wk_1'].astype('str')
race_return_6['wk_2'] = race_return_6['haraimodoshi_sanrentan_1a'].str[2:4]
race_return_6['wk_2'] = race_return_6['wk_2'].astype('int')
race_return_6['wk_2'] = race_return_6['wk_2'].astype('str')
race_return_6['wk_3'] = race_return_6['haraimodoshi_sanrentan_1a'].str[4:6]
race_return_6['wk_3'] = race_return_6['wk_3'].astype('int')
race_return_6['wk_3'] = race_return_6['wk_3'].astype('str')
race_return_6['1'] = race_return_6['wk_1'] + ' → ' + race_return_6['wk_2'] + ' → ' + race_return_6['wk_3']
race_return_6['2'] = race_return_6['haraimodoshi_sanrentan_1b'].astype('int')
#race_return_1['3'] = race_return['haraimodoshi_sanrentan_1c']
race_return_6 = race_return_6.reset_index()
race_return_6 = race_return_6.set_index('key')

race_return_6 = race_return_6[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_6 = race_return_6.rename_axis(None)




In [15]:
race_return_6.head()

,0,1,2
201806010101,３連単,15 → 3 → 6,4060
201806010102,３連単,14 → 9 → 2,2035310
201806010103,３連単,7 → 2 → 11,136180
201806010104,３連単,4 → 6 → 10,16560
201806010105,３連単,9 → 3 → 2,12660


In [25]:
#馬連
race_return_7 = race_return.copy()
race_return_7['key'] = race_return['kaisai_nen'] + race_return['keibajo_code'] + race_return['kaisai_kai'] + race_return['kaisai_nichime'] + race_return['race_bango']
race_return_7['0'] = '馬単'
race_return_7['wk_1'] = race_return_7['haraimodoshi_umaren_1a'].str[0:2]
race_return_7['wk_1'] = race_return_7['wk_1'].astype('int')
race_return_7['wk_1'] = race_return_7['wk_1'].astype('str')
race_return_7['wk_2'] = race_return_7['haraimodoshi_umaren_1a'].str[2:4]
race_return_7['wk_2'] = race_return_7['wk_2'].astype('int')
race_return_7['wk_2'] = race_return_7['wk_2'].astype('str')
race_return_7['1'] = race_return_7['wk_1'] + ' → ' + race_return_7['wk_2']
race_return_7['2'] = race_return_7['haraimodoshi_umaren_1b'].astype('int')
#race_return_1['3'] = race_return['haraimodoshi_umaren_1c']
race_return_7 = race_return_7.reset_index()
race_return_7 = race_return_7.set_index('key')

race_return_7 = race_return_7[[\
                '0',\
                '1',\
                '2'\
 #               '3'\
                ]]
race_return_7 = race_return_7.rename_axis(None)



In [29]:
race_return_all.loc['201806010101']

,0,1,2
201806010101,単勝,15,200
201806010101,複勝,15br3br6,110br140br180
201806010101,馬連,3 - 15,690
201806010101,ワイド,3 - 15br6 - 15br3 - 6,270br350br620
201806010101,３連複,3 - 6 - 15,1290
201806010101,３連単,15 → 3 → 6,4060
201806010101,馬単,3 → 15,690


In [28]:
race_return_all = pd.concat([race_return_1, race_return_2, race_return_3, race_return_4, race_return_5, race_return_6, race_return_7])